In [8]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd

In [4]:
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

driver.get("https://www.allrecipes.com/recipes-a-z-6735880")

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import time

try:
    # Inicializar el driver con opciones adicionales
    options = webdriver.FirefoxOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Firefox(
        service=Service(GeckoDriverManager().install()),
        options=options
    )
    
    # Añadir un tiempo de espera implícito
    driver.implicitly_wait(10)
    
    # Cargar la página con reintento
    max_retries = 3
    for attempt in range(max_retries):
        try:
            driver.get("https://www.allrecipes.com/recipes-a-z-6735880")
            time.sleep(5)  # Esperar 5 segundos para que la página cargue completamente
            break
        except Exception as e:
            print(f"Intento {attempt + 1} falló: {str(e)}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2)

    # Definir XPath para los elementos que contienen los enlaces
    element_xpath = '//div[@class="mntl-alphabetical-list__group"]//a'
    
    # Inicializar conjunto para almacenar enlaces únicos
    enlaces_set = set()
    
    # Esperar a que los elementos estén presentes con reintento
    max_retries = 3
    for attempt in range(max_retries):
        try:
            wait = WebDriverWait(driver, 20)  # Aumentar el tiempo de espera a 20 segundos
            elements = wait.until(
                EC.presence_of_all_elements_located((By.XPATH, element_xpath))
            )
            break
        except Exception as e:
            print(f"Intento {attempt + 1} de encontrar elementos falló: {str(e)}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2)
    
    # Extraer enlaces
    for element in elements:
        try:
            article_link = element.get_attribute("href")
            article_text = element.text.strip()
            if article_link and article_text:  # Solo agregar si ambos existen
                enlaces_set.add((article_text, article_link))
        except Exception as e:
            print(f"Error al procesar enlace: {str(e)}")
            continue
    
    # Crear DataFrame
    df = pd.DataFrame(list(enlaces_set), columns=['categoria', 'enlace'])
    
    # Mostrar las primeras filas del DataFrame
    print("\nPrimeras filas del DataFrame:")
    print(df.head())
    
    # Mostrar el número total de enlaces encontrados
    print(f"\nTotal de enlaces únicos encontrados: {len(df)}")
    csv_name = f'enlaces_recetas.csv'
    df.to_csv(csv_name, index=False, encoding='utf-8-sig')
    print(f'{csv_name} guardado')

except Exception as e:
    print(f"Error general: {str(e)}")

finally:
    try:
        driver.quit()
    except:
        pass


Primeras filas del DataFrame:
             categoria                                             enlace
0            Spaghetti  https://www.allrecipes.com/recipes/505/main-di...
1            Meatballs  https://www.allrecipes.com/recipes/15455/main-...
2               Gumbos  https://www.allrecipes.com/recipes/1426/soups-...
3  Slow Cooker Recipes  https://www.allrecipes.com/recipes/253/everyda...
4             Cobblers  https://www.allrecipes.com/recipes/361/dessert...

Total de enlaces únicos encontrados: 378
enlaces_recetas.csv guardado


In [30]:

from datetime import datetime

# Cargar el CSV existente
df = pd.read_csv('enlaces_recetas.csv')

try:
    # Configuración del driver
    options = webdriver.FirefoxOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Firefox(
        service=Service(GeckoDriverManager().install()),
        options=options
    )
    driver.implicitly_wait(10)
    
    # Crear lista para almacenar los resultados
    resultados = []
    
    # Iterar sobre cada enlace del DataFrame
    for index, row in df.iterrows():
        try:
            print(f"Procesando categoría ({index + 1}/{len(df)}): {row['categoria']}")
            
            # Visitar el enlace de la categoría
            driver.get(row['enlace'])
            time.sleep(3)  # Esperar a que cargue la página
            
            # Encontrar todos los enlaces de recetas
            recipe_xpath = '//a[contains(@class, "mntl-card-list-items")]'
            wait = WebDriverWait(driver, 10)
            recipe_elements = wait.until(
                EC.presence_of_all_elements_located((By.XPATH, recipe_xpath))
            )
            
            # Extraer los enlaces de las recetas
            for recipe in recipe_elements:
                try:
                    recipe_link = recipe.get_attribute('href')
                    if recipe_link:
                        resultados.append({
                            'categoria': row['categoria'],
                            'enlace_categoria': row['enlace'],
                            'enlace_receta': recipe_link
                        })
                except Exception as e:
                    print(f"Error al procesar receta: {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"Error al procesar categoría {row['categoria']}: {str(e)}")
            continue
            
    # Crear nuevo DataFrame con los resultados
    df_recetas = pd.DataFrame(resultados)
    
    # Mostrar información del nuevo DataFrame
    print("\nPrimeras filas del DataFrame de recetas:")
    print(df_recetas.head())
    print(f"\nTotal de recetas encontradas: {len(df_recetas)}")
    
    # Guardar en CSV
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    csv_name = f'enlaces_recetas_detallado_{fecha_actual}.csv'
    df_recetas.to_csv(csv_name, index=False, encoding='utf-8-sig')
    print(f"\nArchivo guardado como: {csv_name}")

except Exception as e:
    print(f"Error general: {str(e)}")

finally:
    try:
        driver.quit()
    except:
        pass

Procesando categoría (1/378): Spaghetti
Procesando categoría (2/378): Meatballs
Procesando categoría (3/378): Gumbos
Procesando categoría (4/378): Slow Cooker Recipes
Procesando categoría (5/378): Cobblers
Procesando categoría (6/378): Frittatas
Procesando categoría (7/378): Leftovers
Procesando categoría (8/378): Filet Mignon
Procesando categoría (9/378): Mediterranean Diet
Procesando categoría (10/378): Chocolate Cakes
Procesando categoría (11/378): Pizza Dough
Procesando categoría (12/378): Cinnamon Rolls
Procesando categoría (13/378): Kolache
Procesando categoría (14/378): Upside-Down Cakes
Procesando categoría (15/378): Pot Pies
Procesando categoría (16/378): Potato Salads
Procesando categoría (17/378): Whoopie Pies
Procesando categoría (18/378): Split Pea Soups
Procesando categoría (19/378): Beef Stroganoff
Procesando categoría (20/378): Turkey
Procesando categoría (21/378): Sugar-Free Recipes
Procesando categoría (22/378): Food Gifts
Procesando categoría (23/378): Chicken and Du

In [ ]:
# Cargar CSV con enlaces
filename = "enlaces_recetas_detallado_2025-01-21.csv"
df_recetas = pd.read_csv(filename)
enlaces_recetas = df_recetas.drop_duplicates('enlace_receta').head(700)

# Inicializar variables
recetas_count = 0
recetas_procesadas = set()
todas_las_recetas = []

try:
   options = webdriver.FirefoxOptions()
   options.add_argument('--disable-gpu')
   options.add_argument('--no-sandbox')
   options.add_argument('--disable-dev-shm-usage')
   
   driver = webdriver.Firefox(
       service=Service(GeckoDriverManager().install()),
       options=options
   )
   driver.implicitly_wait(10)

   total_enlaces = len(enlaces_recetas)

   for index, row in enlaces_recetas.iterrows():
       if row['enlace_receta'] not in recetas_procesadas:
           try:
               porcentaje_faltante = ((total_enlaces - recetas_count) / total_enlaces) * 100
               print(f"Falta: {porcentaje_faltante:.2f}% por procesar", end='\r')
               
               recetas_procesadas.add(row['enlace_receta'])
               driver.get(row['enlace_receta'])
               time.sleep(2)

               try:
                   # [resto del código igual...]
                   titulo = driver.find_element(By.XPATH, '//h1[@class="article-heading text-headline-400"]').text.strip()
                   
                   try:
                       descripcion = driver.find_element(By.XPATH, '//p[@class="article-subheading text-body-100"]').text.strip()
                   except:
                       descripcion = ""

                   prep_time = driver.find_element(By.XPATH, '//div[contains(text(),"Prep Time:")]/following-sibling::div').text.strip()
                   cook_time = driver.find_element(By.XPATH, '//div[contains(text(),"Cook Time:")]/following-sibling::div').text.strip()
                   total_time = driver.find_element(By.XPATH, '//div[contains(text(),"Total Time:")]/following-sibling::div').text.strip()
                   servings = driver.find_element(By.XPATH, '//div[contains(text(),"Servings:")]/following-sibling::div').text.strip()

                   ingredient_elements = driver.find_elements(By.XPATH, '//ul[@class="mm-recipes-structured-ingredients__list"]/li')
                   ingredientes = []
                   for item in ingredient_elements:
                       try:
                           cantidad = item.find_element(By.XPATH, './/span[@data-ingredient-quantity="true"]').text.strip()
                       except:
                           cantidad = ''
                       try:
                           unidad = item.find_element(By.XPATH, './/span[@data-ingredient-unit="true"]').text.strip()
                       except:
                           unidad = ''
                       try:
                           ingrediente = item.find_element(By.XPATH, './/span[@data-ingredient-name="true"]').text.strip()
                       except:
                           ingrediente = ''
                       
                       ingredientes.append(f'{cantidad} {unidad} {ingrediente}'.strip())

                   step_elements = driver.find_elements(By.XPATH, '//ol[@id="mntl-sc-block_1-0"]/li/p[not(contains(@class, "figure-article-caption"))]')
                   pasos = []
                   for i, step in enumerate(step_elements, 1):
                       paso_texto = step.text.strip()
                       if paso_texto:
                           pasos.append(f"Paso {i}: {paso_texto}")

                   receta_info = {
                       'categoria': row['categoria'],
                       'titulo': titulo,
                       'descripcion': descripcion,
                       'tiempo_preparacion': prep_time,
                       'tiempo_cocina': cook_time,
                       'tiempo_total': total_time,
                       'porciones': servings,
                       'ingredientes': '\n'.join(ingredientes),
                       'pasos': '\n'.join(pasos),
                       'url': row['enlace_receta']
                   }
                   
                   todas_las_recetas.append(receta_info)
                   recetas_count += 1

                   # Guardado cada 50 recetas como checkpoint
                   if recetas_count % 50 == 0:
                       df_temp = pd.DataFrame(todas_las_recetas)
                       fecha_actual = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                       df_temp.to_csv(f'recetas_allrecipes_checkpoint_{recetas_count}_{fecha_actual}.csv', 
                                    index=False, encoding='utf-8-sig')

               except Exception as e:
                   continue

           except Exception as e:
               continue

except Exception as e:
   print(f"Error general: {str(e)}")

finally:
   if todas_las_recetas:
       df_final = pd.DataFrame(todas_las_recetas)
       fecha_actual = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
       df_final.to_csv(f'recetas_allrecipes_700_{fecha_actual}.csv', index=False, encoding='utf-8-sig')
   driver.quit()